In [ ]:
from datetime import datetime, timedelta
from pathlib import Path

import dateutil

from ck_fridge_sim.control.control_base import (
    ControllerSetting,
    Disturbance,
    Disturbances,
)
from ck_fridge_sim.simulator.example import (
    configure_example_simulation_from_path,
)
from ck_fridge_sim.solver.solvers import RK12
from ck_fridge_sim.sources.sources import (
    Constant,
    Sinusoid,
    Step,
    SunAngleAltitude,
)
from ck_fridge_sim.thermo_tools.unit_conversions import UnitConversions

In [ ]:
tz = dateutil.tz.gettz("America/Los_Angeles")
start_datetime = datetime(2024, 5, 12, 10, 0, 0, tzinfo=tz)
disturbance_list = [
    Disturbance(
        name="ambient_temp__c",
        source=Sinusoid(
            amplitude=8.0,
            offset=16.0,
            shift_time_str=start_datetime.replace(
                hour=9, minute=0, second=0, microsecond=0
            ).isoformat(),
        ),
    ),
    Disturbance(
        name="ambient_rh__perc",
        source=Sinusoid(
            amplitude=10.0,
            offset=50.0,
            shift_time_str=start_datetime.replace(
                hour=21, minute=0, second=0, microsecond=0
            ).isoformat(),
        ),
    ),
    Disturbance(
        name="machine_room_temp__c",
        source=Step(
            start_value=25.0,
            end_value=15.0,
            step_time_str=start_datetime.replace(
                hour=12, minute=0, second=0, microsecond=0
            ).isoformat(),
        ),
    ),
    Disturbance(
        name="sun_angle_altitude",
        source=SunAngleAltitude(
            latitude__deg=32.5494799,
            longitude__deg=-116.9827151,
        ),
    ),
    Disturbance(
        name="cloud_cover__perc",
        source=Step(
            start_value=0.0,
            end_value=50.0,
            step_time_str=start_datetime.replace(
                hour=14, minute=0, second=0, microsecond=0
            ).isoformat(),
        ),
    ),
]
disturbances = Disturbances(disturbance_list=disturbance_list)

In [ ]:
initial_discharge_pressure__psig = 115
discharge_pressure_setpoint = ControllerSetting(
    name="EC-1|pressure_setpoint",
    controller_name="condenser_fan_controller",
    setting_name="setpoint",
    value=initial_discharge_pressure__psig,
    source=Step(
        start_value=initial_discharge_pressure__psig + 10,
        end_value=initial_discharge_pressure__psig,
        step_time_str=start_datetime.replace(
            hour=13, minute=0, second=0, microsecond=0
        ).isoformat(),
    ),
)

suction_pressure_setpoint = ControllerSetting(
    name="LTR-1|pressure_setpoint",
    controller_name="lss_1",
    setting_name="pi_controller",
    nested_setting_names=["setpoint"],
    value=3,
    source=Step(
        start_value=3,
        end_value=1,
        step_time_str=start_datetime.replace(
            hour=16, minute=0, second=0, microsecond=0
        ).isoformat(),
    ),
)

evaporator_setpoints = [
    ControllerSetting(
        name=f"{evap}|temp_setpoint",
        controller_name=f"evaporator_{i+1}_controller",
        setting_name="setpoint",
        value=-17.0,
        source=Constant(
            value=UnitConversions.fahrenheit_to_celsius(-2 + 0.5 * i),
        ),
    )
    for i, evap in enumerate([f"Evap AU-{i+1}" for i in range(7)])
]

hpr_setpoint = ControllerSetting(
    name="HPR-1|level_setpoint",
    controller_name="hpr_level_control",
    setting_name="setpoint",
    value=33.0,
)

settings = [
    suction_pressure_setpoint,
    discharge_pressure_setpoint,
    hpr_setpoint,
] + evaporator_setpoints

In [ ]:
simulator = configure_example_simulation_from_path(
    process_config_path=Path("simulation_payload/simple_flowsheet_config.json"),
    control_config_path=Path("simulation_payload/simple_control_config.yaml"),
    disturbances=disturbances,
    settings=settings,
    start_datetime=start_datetime,
    end_datetime=start_datetime + timedelta(days=4.0),
    solver=RK12(max_step__s=30, relative_tolerance=0.001),
)

In [ ]:
simulator.process_model.plot_graph(simulator.process_model.current_graph())

In [ ]:
# option to load state from json
simulator.load_state_from_json(
    Path("simulation_payload/process_states.json"),
    Path("simulation_payload/control_states.json"),
)

In [ ]:
solution, measurement_list, actuators_list = simulator.simulate(verbose=True)

In [ ]:
simulator.dump_state_to_json(
    Path("simulation_payload/process_states.json"),
    Path("simulation_payload/control_states.json"),
)

In [ ]:
df = simulator.process_results(solution, measurement_list, actuators_list)
df = df.loc[:, ~df.columns.duplicated()]

In [ ]:
fig = simulator.plot_results(
    df,
    device_names=["dual_cool"],
    point_names=[
        "total_refrigeration_power__kw",
    ],
)
fig.show()

In [ ]:
fig = simulator.plot_results(
    df,
    device_names=["compressor_1", "compressor_2", "compressor_3"],
    point_names=[
        "suction_mass_flow__kg_s",
        "sv_position__perc",
        "compressor_speed__rpm",
    ],
)
fig.show()

In [ ]:
fig = simulator.plot_results(
    df,
    device_names=["condenser_1", "ltr_1", "hpr_1", "liquidKingValve_1"],
    point_names=[
        "pressure__psig",
        "level__perc",
        "fan_speed_1__rpm",
        "fan_speed_2__rpm",
        "ambient_temp__c",
    ],
)
fig.show()

In [ ]:
fig = simulator.plot_results(
    df,
    device_names=[f"evaporator_{i+1}" for i in range(6)] + ["room_1"],
    point_names=["liquid_solenoid_on", "fan_speed_1__rpm", "air_temp__f"],
)
fig.show()